# Time zones 
🌐!

In [33]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
from astropy.time import Time
import pytz

timezone = pytz.timezone('US/Pacific')
print(timezone)

def astropy_time_to_local(timezone, time):
    '''
    Localize ``time`` to the ``timezone``.
    '''
    if not isinstance(time, Time):
        time = Time(time)

    return timezone.localize(time.datetime)

def local_to_astropy_time(local_time):
    '''
    Convert localized time ``local_time`` to astropy Time object in UTC
    '''
    return Time(local_time)

# Present time in UTC
utc_now = datetime.datetime.utcnow()

# Localize to my local time in Pacific timezone
lt = astropy_time_to_local(timezone, utc_now)

# Convert back to UTC in astropy time object
UTC = local_to_astropy_time(lt)

print('Localized datetime {} \n = astropy time {}'.format(lt, UTC))
print('{}, {}'.format(*map(type, [lt, UTC])))

US/Pacific
Localized datetime 2015-07-07 21:13:09.191092-07:00 
 = astropy time 2015-07-08 04:13:09.191092
<type 'datetime.datetime'>, <class 'astropy.time.core.Time'>
